# Change detection by clustering over all fields (pinnotes data)

We have already processed all fields from step 1 to step 5 of the workflow. With saved variables, we analyze them by clustering

In [1]:
%matplotlib widget
import os, time, datetime
import numpy as np
import pandas as pd
import geopandas as gpd
from dtw import *
from preprocess import data_extractor
import operator

import matplotlib as mpl
import matplotlib.dates as mdates
from matplotlib.pyplot import cm
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster, set_link_color_palette

from utilefunc.get_palette import get_palette
from evolution_graph import evolution_graph_to_synopsis

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
# Parameter settings
## segmentation parameters 
scale=5        # controls the number of produced segments as well as their size. Higher scale means less and larger segments. 
sigma=0          # diameter of a Gaussian kernel, used for smoothing the image prior to segmentation.
min_size=5      # minimum size of the segment

## BB selection and graph construction parameters
alpha = 0.4
t1 = 0.1
t2 = 0
direction=1  # 0 : from small to big; 1: from big to small

In [3]:
## general data informations
fields_data_fpath = 'pinnote_anomaly_info/annotations_valid_dates_final.csv'
fields_data = pd.read_csv(fields_data_fpath, sep=';', )   # a list of season field ids
sfd_ids = fields_data['sfd_id'].unique()  # all season field ids to process
data_path = 'data_images_2017_2020/'  # path for image time series, each stocked in a file folder named with the sfd_id

# save paths
save_path_df = 'variables/raster_df/scale_{0}/'.format(scale)
if direction == 0:
    save_path_bb = 'variables/BB_evolution_graph/BB_small2big/scale_{0}_alpha_{1}_t1_{2}_t2_{3}/'.format(scale, alpha, t1, t2)
    save_path_shp = 'variables/sfd_bbs_cover/BB_small2big/scale_{0}_alpha_{1}_t1_{2}_t2_{3}/'.format(scale, alpha, t1, t2)
if direction == 1:
    save_path_bb = 'variables/BB_evolution_graph/BB_big2small/scale_{0}_alpha_{1}_t1_{2}_t2_{3}/'.format(scale, alpha, t1, t2)
    save_path_shp = 'variables/sfd_bbs_cover/BB_big2small/scale_{0}_alpha_{1}_t1_{2}_t2_{3}/'.format(scale, alpha, t1, t2)

## Extract all synopsis 

In [4]:
fld_year_synopsis = []

for sfd_id_choice in sfd_ids:
    year_choice = fields_data[fields_data.sfd_id == sfd_id_choice].year.unique()[0]
    raster_df = pd.read_pickle(save_path_df+'{0}_{1}_scale_{2}_raster_seg_df.pkl'.format(sfd_id_choice,year_choice,scale))
    
    dico_year_synopsis = {}
    dico_year_num_bb = {}
    
    segments_test = raster_df['segments_fz'].iloc[(raster_df.index >= datetime.datetime(year_choice,1,1)) & (raster_df.index <= datetime.datetime(year_choice,12,31))]
    raster_ndvi_numpy_test = raster_df['raster_ndvi_numpy'].iloc[(raster_df.index >= datetime.datetime(year_choice,1,1)) & (raster_df.index <= datetime.datetime(year_choice,12,31))]

    bb_final_list = np.load(save_path_bb+'{0}_{1}_scale_{2}_alpha_{3}_t1_{4}_t2_{5}_final_bb.npy'.format(sfd_id_choice,year_choice,scale,alpha,t1,t2), allow_pickle=True)

    ## Evolution graph of each BB given the (fld_id, year)
    dico_year_synopsis[year_choice] = evolution_graph_to_synopsis(sfd_id_choice, year_choice, bb_final_list, segments_test, raster_ndvi_numpy_test, alpha, t1, t2)

    fld_year_synopsis.append(dico_year_synopsis)
fld_year_synopsis_df = pd.DataFrame(fld_year_synopsis, index=sfd_ids)

In [5]:
def func(pct, allvals):
    absolute = int(round(pct/100.*np.sum(allvals)))
    return "{:.1f}%\n({:d})".format(pct, absolute)

data = fields_data.groupby('year').count()['sfd_id']
labels = fields_data.groupby('year').count().index
explode = np.ones((len(labels)))*0.1

plt.figure(figsize=(6,4))
wedges, texts, autotexts = plt.pie(data, autopct=lambda pct: func(pct, data), textprops=dict(color="k"), explode=explode)
plt.legend(wedges, labels, title="Year", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
plt.setp(autotexts, size=8, weight="bold")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
data = fields_data.groupby('croptype').count()['spn_id']
labels = fields_data.groupby('croptype').count().index
explode = np.ones((len(labels)))*0.1

plt.figure(figsize=(10,6))
wedges, texts, autotexts = plt.pie(data, autopct=lambda pct: func(pct, data), textprops=dict(color="k"), explode=explode)
plt.legend(wedges, labels, title="Crop type", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
plt.setp(autotexts, size=8, weight="bold")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
year_choice = 2018
data = fields_data[fields_data.year == year_choice].groupby('croptype').count()['spn_id']
labels = fields_data[fields_data.year == year_choice].groupby('croptype').count().index
explode = np.ones((len(labels)))*0.1
plt.figure(figsize=(10,6))
wedges, texts, autotexts = plt.pie(data, labels=labels, autopct=lambda pct: func(pct, data), textprops=dict(color="k"), explode=explode)
plt.setp(autotexts, size=8, weight="bold")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fields_data.groupby(['year', 'croptype']).count().index

In [ ]:
data = fields_data.groupby(['year', 'croptype']).count()['sfd_id']
labels = fields_data.groupby(['year', 'croptype']).count().index
plt.figure()
plt.bar(range(len(labels)), data)
plt.xticks(ticks=range(len(labels)), labels=labels, rotation=90)
plt.show()

### Choose synopsis by (year, croptype) for clustering

In [5]:
year_choice = 2018
crop_type = 'Corn'

all_dates = []

n_synopsis = 0
for sfd_id_choice in sfd_ids:
    if type(fld_year_synopsis_df.loc[sfd_id_choice][year_choice]) != type(np.nan) and fields_data[fields_data.sfd_id == sfd_id_choice].croptype.values[0] == crop_type:
        all_dates+=[t.dayofyear for x in fld_year_synopsis_df.loc[sfd_id_choice][year_choice] for t in x[0]]
        n_synopsis += len(fld_year_synopsis_df.loc[sfd_id_choice][year_choice])

uni_dates = np.unique(all_dates)

############################
all_synopsis_list = []
all_date_list = []
data_uniform_array = np.ones((n_synopsis, len(uni_dates)))*1000

update = 0
syn_label = []

for sfd_id_choice in sfd_ids:
    if type(fld_year_synopsis_df.loc[sfd_id_choice][year_choice]) != type(np.nan) and fields_data[fields_data.sfd_id == sfd_id_choice].croptype.values[0] == crop_type:
        date_list = []
        synopsis_list = []
        bb_ids = []
        for x in fld_year_synopsis_df.loc[sfd_id_choice][year_choice]:
            date_list.append(x[0])
            synopsis_list.append(x[1])
            bb_ids.append(str(x[2][0]) + '_' + str(x[2][1]))
        helper = pd.DataFrame({'date':[pd.to_datetime(uni_dates[i]-1, unit='D', origin=str(year_choice)) for i in range(len(uni_dates))]})
        
        for i in range(len(date_list)):
            df = pd.DataFrame({'date':date_list[i], 'val':synopsis_list[i]})
            df = pd.merge(df, helper, on='date', how='outer').sort_values('date')
            df.interpolate(method='linear', limit_area='inside', inplace=True)
            data_uniform_array[update] = df['val'].values
            update += 1
            
            syn_label.append(str(sfd_id_choice)+':'+bb_ids[i])

        all_date_list += date_list
        all_synopsis_list += synopsis_list

data_uniform_array[data_uniform_array==1000] = np.nan
print(f'n_synopsis size : {n_synopsis}')

n_synopsis size : 280


In [7]:
clustered_sfd_ids = list(set([el.split(':')[0] for el in syn_label]))
clustered_sfd_ids

['102112829',
 '102008580',
 '102006104',
 '102252397',
 '102536577',
 '102403591',
 '102009130',
 '102328550',
 '102203873',
 '102424640',
 '102103879']

In [6]:
def dtw_normalized_dist(a, b):
    # find dates where both of the two time series have data
    common = ~np.isnan(a) & ~np.isnan(b)
    a_c = a[common]
    b_c = b[common]
    if a_c.size == 0 or b_c.size == 0: # if one of them is empty, it means they have non dates in common
        distance = 1   #10
    else:
        # delete NANs in the beginning and the end of the time series
        a = a[~np.isnan(a)]
        b = b[~np.isnan(b)]
        alignment = dtw(a, b, keep_internals=True)
        distance = alignment.normalizedDistance  # normalised by the sum of lengths of the two series
    return distance

### DTW distance matrix

In [7]:
matrix_dist = np.zeros((data_uniform_array.shape[0], data_uniform_array.shape[0]))
for i in range(data_uniform_array.shape[0]):
    for j in range(i, data_uniform_array.shape[0]):
        matrix_dist[i][j] = dtw_normalized_dist(data_uniform_array[i], data_uniform_array[j])
        matrix_dist[j][i] = matrix_dist[i][j]

### Method 1 - OPTICS

In [8]:
from sklearn.cluster import OPTICS, cluster_optics_dbscan

In [9]:
clust = OPTICS(min_samples=10, xi=.05, cluster_method='xi', metric='precomputed').fit(matrix_dist)

# labels_050 = cluster_optics_dbscan(reachability=clust.reachability_,
#                                    core_distances=clust.core_distances_,
#                                    ordering=clust.ordering_, eps=0.5)
# labels_200 = cluster_optics_dbscan(reachability=clust.reachability_,
#                                    core_distances=clust.core_distances_,
#                                    ordering=clust.ordering_, eps=2)



In [10]:
space = np.arange(len(matrix_dist))
reachability = clust.reachability_[clust.ordering_]
labels = clust.labels_[clust.ordering_]

plt.figure(figsize=(10,6))
colors = ['g.', 'r.', 'b.', 'y.', 'c.', 'o.']
# colors = iter([plt.cm.tab10(i) for i in range(10)])
for klass, color in zip(range(0, 5), colors):
    Xk = space[labels == klass]
    Rk = reachability[labels == klass]
    plt.plot(Xk, Rk, color, alpha=0.3)
plt.plot(space[labels == -1], reachability[labels == -1], 'k.', alpha=0.3)
plt.plot(space, np.full_like(space, 0.01, dtype=float), 'k-', alpha=0.5)
plt.plot(space, np.full_like(space, 0.015, dtype=float), 'k-.', alpha=0.5)
plt.ylabel('Reachability (epsilon distance)')
plt.title('Reachability Plot')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Method 2 - DBSCAN

In [12]:
from sklearn.cluster import DBSCAN

In [20]:
clustering = DBSCAN(eps=0.015, min_samples=10, metric='precomputed').fit(matrix_dist)

In [21]:
_, ax1 = plt.subplots(figsize=(10,6))
ax1.grid(True)

colors = iter([plt.cm.tab10(i) for i in range(10)])
color = dict()
cluster_color = dict()
for label in np.unique(clustering.labels_):
    color[label] = next(colors)
    cluster_color[label] = np.where(clustering.labels_ == label)[0]

for i in range(data_uniform_array.shape[0]):
    ax1.plot(helper, data_uniform_array[i], color=color[clustering.labels_[i]])

box = ax1.get_position()
ax1.set_position([box.x0, box.y0, box.width*0.8, box.height])
legend_element = [plt.Line2D([0], [0], color=c, linestyle='-') for c in color.values()]
legend_label = ['outliers'] + ['cluster_'+str(i+1) for i in range(len(color.values())-1)]

ax1.legend(legend_element, legend_label, bbox_to_anchor=(0.75, 0.7), bbox_transform=plt.gcf().transFigure)
plt.title("Clustering results of the synopsis vectors of evolution graphs of all fields")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# Séparer les courbes de synopsis par cluster

_, ax = plt.subplots(figsize=(5*2,5*3), nrows=4, ncols=2)
axis = ax.flatten()

axis[0].grid(True)
for i in range(len(all_synopsis_list)):
    axis[0].plot(all_date_list[i], all_synopsis_list[i], color=color[clustering.labels_[i]])
axis[0].set_ylim([0, 1])
plt.setp(axis[0].get_xticklabels(), rotation=90, fontsize=8)

x_l = mdates.date2num(helper.iloc[0].values)
x_r = mdates.date2num(helper.iloc[-1].values)
for idx, (k, v) in enumerate(color.items()):
    l = np.where(clustering.labels_ == k)[0]
    axis[idx+1].grid(True)
    for j in l:
        axis[idx+1].plot(all_date_list[j], all_synopsis_list[j], color=v)
    axis[idx+1].set_xlim([x_l, x_r])
    axis[idx+1].set_ylim([0, 1])
    axis[idx+1].set_title(legend_label[k+1])
    plt.setp(axis[idx+1].get_xticklabels(), rotation=90, fontsize=8)

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
data_uniform_array[clustering.labels_ == 0].shape

In [ ]:
# Z = linkage(data_uniform_array[clustering.labels_ == -1], method='complete', metric=dtw_normalized_dist)

### Spectral clustering

In [ ]:
from sklearn.cluster import SpectralClustering

In [ ]:
clustering = SpectralClustering(n_clusters=20, 
                                assign_labels='discretize',
                                random_state=0,
                                affinity='precomputed').fit(matrix_dist)

In [ ]:
# Séparer les courbes de synopsis par cluster
colors = iter([plt.cm.tab20(i) for i in range(20)])
color = dict()
cluster_color = dict()
for label in np.unique(clustering.labels_):
    color[label] = next(colors)
    cluster_color[label] = np.where(clustering.labels_ == label)[0]
    
nrows=7
ncols=3
_, ax = plt.subplots(figsize=(4*ncols,4*nrows), nrows=nrows, ncols=ncols)
axis = ax.flatten()

axis[0].grid(True)
for i in range(len(all_synopsis_list)):
    axis[0].plot(all_date_list[i], all_synopsis_list[i], color=color[clustering.labels_[i]])
axis[0].set_ylim([0, 1])
plt.setp(axis[0].get_xticklabels(), rotation=90, fontsize=8)

x_l = mdates.date2num(helper.iloc[0].values)
x_r = mdates.date2num(helper.iloc[-1].values)
for idx, (k, v) in enumerate(color.items()):
    l = np.where(clustering.labels_ == k)[0]
    axis[idx+1].grid(True)
    for j in l:
        axis[idx+1].plot(all_date_list[j], all_synopsis_list[j], color=v)
    axis[idx+1].set_xlim([x_l, x_r])
    axis[idx+1].set_ylim([0, 1])
    plt.setp(axis[idx+1].get_xticklabels(), rotation=90, fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
#### old version of DTW distance for synopsis

def dtw_normalized_dist(a, b):
    # prendre les dates où les deux séries ont toutes les deux des données
    common = ~np.isnan(a) & ~np.isnan(b)
    a = a[common]
    b = b[common]
    # si l'un des deux est nul, c'est à dire qu'il n'y a pas de dates en communes
    if a.size == 0 or b.size == 0:
        distance = 1   #10
    else:
        # enlever les patterns du début et/ou de la fin de la série avec des NANs
        a = a[~np.isnan(a)]
        b = b[~np.isnan(b)]

        alignment = dtw(a, b, keep_internals=True)
        distance = alignment.normalizedDistance
    return distance

### CAH

In [12]:
def get_cluster_color(dendrogram):
    cluster_idxs = dict()
    for c, pi in zip(dendrogram['color_list'], dendrogram['icoord']):
        l = []
        for leg in pi[1:3]:
            i = (leg - 5.0) / 10.0
            if abs(i - int(i)) < 1e-5:
                l.append(int(i))
        cluster_idxs[c] = cluster_idxs.get(c, []) + l
    cluster_classes = dict()
    for c, l in cluster_idxs.items():
        i_l = [dendrogram['ivl'][i] for i in l]
        cluster_classes[c] = np.unique(i_l)
    return cluster_classes

In [43]:
plt.close('all')

In [ ]:
dates_dispo = np.ones(data_uniform_array.shape)
dates_dispo[np.isnan(data_uniform_array)] = 0
dates_dispo

In [ ]:
data_diff = np.diff(data_uniform_array)
day_diff = [(helper['date'].iloc[i]-helper['date'].iloc[i-1]).days for i in range(1, len(helper['date']))]
grad = np.zeros(data_diff.shape)
for i in range(len(day_diff)):
    grad[:,i] = data_diff[:, i]/day_diff[i]

In [ ]:
bbs_size = []
for x in np.array(syn_label).tolist():
    sfd_id_choice = x.split(':')[0]
    bb_img = int(x.split(':')[1].split('_')[0])
    bb_seg = int(x.split(':')[1].split('_')[1])
    bb_final_list = np.load(bb_save_path+str(sfd_id_choice)+'_'+str(year_choice)+'_alpha_'+str(alpha)+'_t1_'+str(t1)+'_t2_'+str(t2)+'_final_bb.npy', allow_pickle=True)
    bb_size = bb_final_list[(bb_final_list[:,0]==bb_img) & (bb_final_list[:,1]==bb_seg), 2].all()
    raster_df = pd.read_pickle(raster_df_save_path + str(sfd_id_choice) + '_' + str(year_choice) + '_raster_seg_df.pkl')
    bb_cover_rate = bb_size/raster_df['raster_ndvi_numpy'][0].size
    bbs_size.append([bb_cover_rate])
bbs_size = np.array(bbs_size)

In [89]:
data_uniform_array_weighted = bbs_size * data_uniform_array

NameError: name 'bbs_size' is not defined

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
# Z = linkage(dates_dispo, method='single', metric='cityblock')
# Z = linkage(data_uniform_array_weighted, method='complete', metric=dtw_normalized_dist)
# Z = linkage(grad, method='average', metric=dtw_normalized_dist)
# np.set_printoptions(suppress=True)

In [68]:
method = 'average'
Z = linkage(data_uniform_array, method=method, metric=dtw_normalized_dist)

In [52]:
cmap = cm.gist_rainbow(np.linspace(0, 1, 50))
set_link_color_palette([mpl.colors.rgb2hex(rgb[:3]) for rgb in cmap])

### To choose a better threshold for dendrogram cut

In [ ]:
xthreshold = np.arange(0.1,0,-0.005)

intra_std = []
intra_mean = []
inter = []
num_cluster = []
x = []
plt.figure()
for threshold in xthreshold:
    d = dendrogram(Z, orientation='right', truncate_mode='level', p=n_synopsis, color_threshold=threshold, above_threshold_color='grey')

    cluster_color = get_cluster_color(d)
    if len(cluster_color.keys())>50:
        break
    else:
        centers = []
        # inertie intra d'un cluster
        intra_k = []
        for idx, (k, v) in enumerate(cluster_color.items()):
            l = [int(i) for i in v]

            if len(l) != 0:
                center = np.nanmean(data_uniform_array[l, :], axis=0)
                centers.append(center)

                inertie_intra = 0
                for j in l:
                    inertie_intra += dtw_normalized_dist(data_uniform_array[j, :], center)**2
                inertie_intra = inertie_intra/len(l)
            else:
                continue
            intra_k.append(inertie_intra)
    #         print(f'inertie intra cluster : {k} - {inertie_intra}')
        intra_std.append(np.nanstd(intra_k))
        intra_mean.append(np.nanmean(intra_k))

        # inertie inter des clusters (variance des centres des clusters)
        inertie_inter = 0
        c = np.nanmean(np.array(centers), axis=0)
        for center in centers:
            inertie_inter += dtw_normalized_dist(c, center)**2
            inertie_inter = inertie_inter/len(centers)
        inter.append(inertie_inter)
        num_cluster.append(len(centers))
#     print(f'inertie inter cluster : {inertie_inter}')
        x.append(threshold)
plt.close()

In [53]:
plt.close('all')

In [ ]:
fig = plt.figure(figsize=(10,7))
plt.grid(True)
ax1 = fig.add_subplot(111)

ax2 = ax1.twinx()
# ax2.plot(x, num_cluster, '--g', label='Nombre de clusters')
# ax2.set_ylabel('Nombre de clusters')

ax1.plot(x, inter, label='Inertie inter-cluster')
ax1.set_ylabel('Inertie')

ax1.errorbar(x, intra_mean, yerr=intra_std, fmt='--.r', ecolor='grey', elinewidth=2, capsize=2, label='Inertie intra cluster')
ax1.plot(x, intra_std, '--o', label='Std intra cluster')
ax2.plot(x, [intra_mean[i]/inter[i] for i in range(len(x))], '--r', label='Ratio intra/inter')
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

plt.show()

#### Optimal threshold

In [69]:
threshold = 0.015

labels = fcluster(Z, t=threshold, criterion='distance')

# set_link_color_palette(None)

plt.figure(figsize=(10, 6.5))
d = dendrogram(Z, orientation='right', truncate_mode='level', p=n_synopsis, color_threshold=threshold, above_threshold_color='grey')

plt.axvline(x=threshold, c='r', lw=2, linestyle='dashed')
plt.title(f"Dendrogram result of fields with {crop_type} in {year_choice} : {method}, cut level at {threshold}")
plt.show()

cluster_color = get_cluster_color(d)
label_color = {label:k for k,v in cluster_color.items() for label in v}
len(cluster_color.keys())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

24

In [13]:
plt.close('all')

In [ ]:
# _, ax1 = plt.subplots(figsize=(10,6))
# ax1.grid(True)
# for i in range(len(all_synopsis_list)):
#     ax1.plot(all_date_list[i], all_synopsis_list[i], color=label_color[str(i)])

# box = ax1.get_position()
# ax1.set_position([box.x0, box.y0, box.width*0.8, box.height])

# legend_element = [plt.Line2D([0], [0], color=c, linestyle='-') for c in cluster_color.keys()]
# legend_label = [i for i in cluster_color.keys()]

# ax1.legend(legend_element, legend_label, bbox_to_anchor=(0.75, 0.7), bbox_transform=plt.gcf().transFigure)
# plt.title(f"Clustering results of the synopsis vectors of evolution graphs of all fields in {year_choice}")
# plt.show()

In [70]:
nrows=9
ncols=3
_, ax = plt.subplots(figsize=(4*ncols,4*nrows), nrows=nrows, ncols=ncols)
axis = ax.flatten()

x_l = mdates.date2num(helper.iloc[0].values)
x_r = mdates.date2num(helper.iloc[-1].values)

axis[0].grid(True)
for i in range(len(all_synopsis_list)):
    axis[0].plot(all_date_list[i], all_synopsis_list[i], color=label_color[str(i)])
#     axis[0].plot(helper, data_uniform_array[i], color=label_color[str(i)])

axis[0].set_xlim([x_l, x_r])
axis[0].set_ylim([0, 1])
axis[0].set_title(f'all synopsis with cluster color')
plt.setp(axis[0].get_xticklabels(), rotation=90, fontsize=8)

for idx, (k, v) in enumerate(cluster_color.items()):
    l = np.unique([int(i) for i in v])
    axis[idx+1].grid(True)
    for j in l:
        axis[idx+1].plot(all_date_list[j], all_synopsis_list[j], color=k)
#         axis[idx+1].plot(helper, data_uniform_array[j], color=k)
    axis[idx+1].set_xlim([x_l, x_r])
    axis[idx+1].set_ylim([0, 1])
    axis[idx+1].set_title(f'{idx} - {k} ({len(l)} BBs)')
    plt.setp(axis[idx+1].get_xticklabels(), rotation=90, fontsize=8)
plt.tight_layout()
plt.suptitle(f'Clustering results by {method} at cut level {threshold} : fields with {crop_type} in {year_choice}', y=1)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
fig = plt.figure(figsize=(10, 6.5))
d_id = dendrogram(Z, orientation='right', truncate_mode='level', p=n_synopsis, labels=syn_label, color_threshold=threshold, above_threshold_color='grey')


plt.axvline(x=threshold, c='r', lw=2, linestyle='dashed')
plt.title(f"Dendrogram result of fields with {crop_type} in {year_choice} : cut level at {threshold}")
plt.show()
cluster_color_id = get_cluster_color(d_id)
label_color_id = {label:k for k,v in cluster_color_id.items() for label in v}

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
cluster_anomaly_bb = {k:[] for k in cluster_color_id.keys()}
cluster_anomaly_precision = {k:[] for k in cluster_color_id.keys()}
cluster_anomaly_recall = {k:[] for k in cluster_color_id.keys()}
cluster_anomaly_p_r = {k:[] for k in cluster_color_id.keys()}

sfd_bb_precision = dict()
sfd_bb_recall = dict()
sfd_bb_p_r = dict()
sfd_bb_F = dict()

for sfd_id_choice in clustered_sfd_ids:
#     year_choice = annotations_info[annotations_info.sfd_id == sfd_id_choice].year.values[0]
    annotation_df = pd.read_pickle(f'variables/annotation_df/{sfd_id_choice}_{year_choice}_annotation_df.pkl')

    raster_df = pd.read_pickle(save_path_df+'{0}_{1}_scale_{2}_raster_seg_df.pkl'.format(sfd_id_choice,year_choice,scale))
    segments_test = raster_df['segments_fz'].iloc[(raster_df.index >= datetime.datetime(year_choice,1,1)) & (raster_df.index <= datetime.datetime(year_choice,12,31))]
    raster_ndvi_numpy_test = raster_df['raster_ndvi_numpy'].iloc[(raster_df.index >= datetime.datetime(year_choice,1,1)) & (raster_df.index <= datetime.datetime(year_choice,12,31))]

    bb_final_list = np.load(save_path_bb+'{0}_{1}_scale_{2}_alpha_{3}_t1_{4}_t2_{5}_final_bb.npy'.format(sfd_id_choice,year_choice,scale,alpha,t1,t2), allow_pickle=True)

    criterion_1 = []
    criterion_2 = []
    criterion_3 = []
    criterion_4 = []
    n = []
    anomaly_bb_ids = []
    for i in range(bb_final_list.shape[0]):
        dico_bb = dict()
        for el in bb_final_list[i,4]:
            dico_bb[el[0]] = dico_bb.get(el[0], []) + [el[1]]
        dico_bb[bb_final_list[i,0]] = [bb_final_list[i,1]]
        dico_bb = dict(sorted(dico_bb.items(), key=operator.itemgetter(0)))
        commun_dates = segments_test.index[list(dico_bb.keys())].intersection(annotation_df.index)
        commun_dates_idx = segments_test.index.get_indexer(commun_dates)
        dates_in_graph = len(commun_dates)/len(annotation_df.index)*100
        graph_not_anomaly = (1-len(commun_dates)/len(dico_bb))*100

        precision_avg = []
        recall_avg = []
        F_measure_avg = []
        accuracy_avg = []
        p_r_avg = []
        for idx, j in enumerate(commun_dates_idx):
            bb_cover = np.ma.zeros(segments_test.iloc[j].shape)
            for el in dico_bb[j]:
                bb_cover[segments_test.iloc[j] == el] = 1
            bb_cover.mask = raster_ndvi_numpy_test.iloc[j].mask
            annotation = annotation_df[annotation_df.index == commun_dates[idx]]['annotes_numpy'][0]
            TP = (bb_cover*annotation).sum()
            FP = (bb_cover*(annotation == 0)).sum()
            TN = ((bb_cover == 0)*(annotation == 0)).sum()
            FN = ((bb_cover == 0)*annotation).sum()
            
            precision = TP/(TP+FP) 
            recall = TP/(TP+FN)
            F_measure = 2*precision*recall/(precision+recall)
            accuracy = (TP+TN)/(TP+TN+FP+FN)
            p_r = precision*recall
            
            precision_avg.append(precision)
            recall_avg.append(recall)
            F_measure_avg.append(F_measure)
            accuracy_avg.append(accuracy)
            p_r_avg.append(p_r)
        # performance over annotation for each BB
        precision_avg = np.nansum(precision_avg)/len(annotation_df)
        recall_avg = np.nansum(recall_avg)/len(annotation_df)
        F_measure_avg = np.nansum(F_measure_avg)/len(annotation_df)
        accuracy_avg = np.nansum(accuracy_avg)/len(annotation_df)
        p_r_avg = np.nansum(p_r_avg)/len(annotation_df)
        
        if precision_avg > 0 and recall_avg > 0:  # if BB covers the annotated pixels
            criterion_1.append(precision_avg)
            criterion_2.append(recall_avg)
            criterion_3.append(p_r_avg)
            criterion_4.append(F_measure_avg)
            n.append(i)
            anomaly_bb_ids.append(f'{bb_final_list[i,0]}_{bb_final_list[i,1]}')

#     sfd_bb_precision[sfd_id_choice] = sfd_bb_precision.get(sfd_id_choice, []) + criterion_1
#     sfd_bb_recall[sfd_id_choice] = sfd_bb_recall.get(sfd_id_choice, []) + criterion_2
#     sfd_bb_p_r[sfd_id_choice] = sfd_bb_p_r.get(sfd_id_choice, []) + criterion_3
#     sfd_bb_F[sfd_id_choice] = sfd_bb_F.get(sfd_id_choice, []) + criterion_4
    
    # save sfd:bb_id, corresponding precision, recall by cluster
    sfd_anomaly_bb = ['{0}:{1}'.format(str(sfd_id_choice), el) for el in anomaly_bb_ids]
    for idx, el in enumerate(sfd_anomaly_bb):
        cluster_anomaly_bb[label_color_id[el]] += [el]    
        cluster_anomaly_precision[label_color_id[el]] += [criterion_1[idx]]
        cluster_anomaly_recall[label_color_id[el]] += [criterion_2[idx]]
        cluster_anomaly_p_r[label_color_id[el]] += [criterion_2[idx]]

<ipython-input-72-ea2d01fdd8ff>:56: RuntimeWarning: invalid value encountered in double_scalars
  F_measure = 2*precision*recall/(precision+recall)
<ipython-input-72-ea2d01fdd8ff>:56: RuntimeWarning: invalid value encountered in double_scalars
  F_measure = 2*precision*recall/(precision+recall)
<ipython-input-72-ea2d01fdd8ff>:56: RuntimeWarning: invalid value encountered in double_scalars
  F_measure = 2*precision*recall/(precision+recall)
<ipython-input-72-ea2d01fdd8ff>:56: RuntimeWarning: invalid value encountered in double_scalars
  F_measure = 2*precision*recall/(precision+recall)
<ipython-input-72-ea2d01fdd8ff>:56: RuntimeWarning: invalid value encountered in double_scalars
  F_measure = 2*precision*recall/(precision+recall)
<ipython-input-72-ea2d01fdd8ff>:56: RuntimeWarning: invalid value encountered in double_scalars
  F_measure = 2*precision*recall/(precision+recall)
<ipython-input-72-ea2d01fdd8ff>:56: RuntimeWarning: invalid value encountered in double_scalars
  F_measure = 2*

In [80]:
dico_cluster_anomaly_rate = {k:len(cluster_anomaly_bb[k])/len(v)*100 for k,v in cluster_color_id.items() if len(v)>0}
dico_cluster_anomaly_type = {k:[fields_data[fields_data.sfd_id == int(el.split(':')[0])].anomaly_class.values[0] for el in v] 
                             if len(v) > 0 else [] for k,v in cluster_anomaly_bb.items()}

# nrows=8
# ncols=3
_, ax = plt.subplots(figsize=(4*ncols,4*nrows), nrows=nrows, ncols=ncols)
axis = ax.flatten()

x_l = mdates.date2num(helper.iloc[0].values)
x_r = mdates.date2num(helper.iloc[-1].values)

axis[0].grid(True)
for i in range(len(all_synopsis_list)):
    axis[0].plot(all_date_list[i], all_synopsis_list[i], color=label_color[str(i)])
#     axis[0].plot(helper, data_uniform_array[i], color=label_color[str(i)])
axis[0].set_xlim([x_l, x_r])
axis[0].set_ylim([0, 1])
axis[0].set_title(f'all synopsis with cluster color', fontsize=8)
plt.setp(axis[0].get_xticklabels(), rotation=90, fontsize=8)
plt.setp(axis[0].get_yticklabels(), fontsize=8)

for idx, (k, v) in enumerate(cluster_color.items()):
    
    l = np.unique([int(i) for i in v])
    axis[idx+1].grid(True)
    for j in l:
        axis[idx+1].plot(all_date_list[j], all_synopsis_list[j], color=k)
#         axis[idx+1].plot(helper, data_uniform_array[j], color=k)
    axis[idx+1].set_xlim([x_l, x_r])
    axis[idx+1].set_ylim([0, 1])
    if len(set(dico_cluster_anomaly_type[k])) > 0:
        a = round(dico_cluster_anomaly_rate[k],2)
        b = set(dico_cluster_anomaly_type[k])
        c = round(np.mean(cluster_anomaly_precision[k]),2)
        d = round(np.max(cluster_anomaly_precision[k]),2)
        e = round(np.mean(cluster_anomaly_recall[k]),2)
        f = round(np.max(cluster_anomaly_recall[k]),2)
        title = f'{idx} - {k} ({len(l)} BBs)\n {a}% anomaly BBs with type {b}\n precision: mean={c} max={d} recall: mean={e} max={f}'

        axis[idx+1].set_title(title, fontsize=8)
    else:
        axis[idx+1].set_title(f'{idx} - {k} ({len(l)} BBs)\n no anomaly BBs', fontsize=8)

    plt.setp(axis[idx+1].get_xticklabels(), rotation=90, fontsize=8)
    plt.setp(axis[idx+1].get_yticklabels(), fontsize=8)

plt.tight_layout()
plt.suptitle(f'Clustering results by {method} at cut level {threshold} : fields with {crop_type} in {year_choice}', y=1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IndexError: index 24 is out of bounds for axis 0 with size 24

In [73]:
plt.figure(figsize=(12,8))
plt.subplot(1,2,1)
plt.grid(True)
plt.boxplot(sfd_bb_precision.values(), labels=sfd_bb_precision.keys())
data_x = [i for i,k in enumerate(sfd_bb_precision.keys()) if len(sfd_bb_precision[k]) > 0 for el in sfd_bb_precision[k]]
data_y = [el for v in list(sfd_bb_precision.values()) if len(v) > 0 for el in v]
for x, y in zip(data_x, data_y):
    plt.scatter(x+1, y, alpha=0.4, c='r')
plt.xlabel('sfd_id')
plt.ylabel('average precision of all annotations per BB')
plt.xticks(rotation=45)
plt.ylim(top=1.1)

plt.subplot(1,2,2)
plt.grid(True)
plt.boxplot(sfd_bb_recall.values(), labels=sfd_bb_recall.keys())
data_x = [i for i,k in enumerate(sfd_bb_recall.keys()) if len(sfd_bb_recall[k]) > 0 for el in sfd_bb_recall[k]]
data_y = [el for v in list(sfd_bb_recall.values()) if len(v) > 0 for el in v]
for x, y in zip(data_x, data_y):
    plt.scatter(x+1, y, alpha=0.4, c='r')
plt.xlabel('sfd_id')
plt.ylabel('average recall of all annotations per BB')
plt.xticks(rotation=45)
plt.ylim(top=1.1)

plt.suptitle(f'Performance over fields with {crop_type} in {year_choice}')
plt.tight_layout()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: Dimensions of labels and X must be compatible

In [89]:
dico_cluster_anomaly_rate = {k:len(cluster_anomaly_bb[k])/len(v)*100 for k,v in cluster_color_id.items() if len(v)>0}
dico_cluster_anomaly_type = {k:[fields_data[fields_data.sfd_id == int(el.split(':')[0])].anomaly_class.values[0] for el in v] 
                             if len(v) > 0 else [] for k,v in cluster_anomaly_bb.items()}

nrows=8
ncols=3
_, ax = plt.subplots(figsize=(4*ncols,4*nrows), nrows=nrows, ncols=ncols)
axis = ax.flatten()

x_l = mdates.date2num(helper.iloc[0].values)
x_r = mdates.date2num(helper.iloc[-1].values)

# axis[0].grid(True)
# for i in range(len(all_synopsis_list)):
#     axis[0].plot(all_date_list[i], all_synopsis_list[i], color=label_color[str(i)])
# #     axis[0].plot(helper, data_uniform_array[i], color=label_color[str(i)])
# axis[0].set_xlim([x_l, x_r])
# axis[0].set_ylim([0, 1])
# axis[0].set_title(f'all synopsis with cluster color', fontsize=8)
# plt.setp(axis[0].get_xticklabels(), rotation=90, fontsize=8)
# plt.setp(axis[0].get_yticklabels(), fontsize=8)

for idx, (k, v) in enumerate(cluster_color.items()):
    idx -= 1
    l = np.unique([int(i) for i in v])
    axis[idx+1].grid(True)
    for j in l:
        axis[idx+1].plot(all_date_list[j], all_synopsis_list[j], color='tab:cyan')
#         axis[idx+1].plot(helper, data_uniform_array[j], color=k)
    axis[idx+1].set_xlim([x_l, x_r])
    axis[idx+1].set_ylim([0, 1])
    if len(set(dico_cluster_anomaly_type[k])) > 0:
        a = round(dico_cluster_anomaly_rate[k],2)
        b = set(dico_cluster_anomaly_type[k])
        c = round(np.mean(cluster_anomaly_precision[k]),2)
        d = round(np.max(cluster_anomaly_precision[k]),2)
        e = round(np.mean(cluster_anomaly_recall[k]),2)
        f = round(np.max(cluster_anomaly_recall[k]),2)
#         title = f'{idx} - {k} ({len(l)} BBs)\n {a}% anomaly BBs with type {b}\n precision: mean={c} max={d} recall: mean={e} max={f}'
        title = f'cluster ({idx+1}) - {len(l)} BBs avec {a}% anormales \n classe {b}\n précision: moy={c}, max={d} \n rappel: moy={e}, max={f}'

        axis[idx+1].set_title(title, fontsize=12)
    else:
#         axis[idx+1].set_title(f'{idx} - {k} ({len(l)} BBs)\n no anomaly BBs', fontsize=8)
        axis[idx+1].set_title(f'cluster ({idx+1}) - {len(l)} BBs\n aucune BB anormale', fontsize=12)

    plt.setp(axis[idx+1].get_xticklabels(), rotation=90, fontsize=12)
    plt.setp(axis[idx+1].get_yticklabels(), fontsize=12)

plt.tight_layout()
# plt.suptitle(f'Clustering results by {method} at cut level {threshold} : fields with {crop_type} in {year_choice}', y=1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
plt.close('all')

In [18]:
import collections
import math

### purity

In [62]:
dict_multi_labels = dict()
for k,v in cluster_color_id.items():
    for el in v:
        if el in cluster_anomaly_bb[k]:
            dict_multi_labels[k] = dict_multi_labels.get(k, []) + [fields_data[fields_data.sfd_id == int(el.split(':')[0])].anomaly_class.values[0]]
        else:
            dict_multi_labels[k] = dict_multi_labels.get(k, []) + ['normal']
multi_purity = dict()
for k,v in dict_multi_labels.items():
    multi_purity[k] = collections.Counter(v).most_common(1)[0][1]
purity = sum(list(multi_purity.values()))/n_synopsis
print(f'multi class purity : {purity}')

multi class purity : 0.75


In [63]:
dict_binary_labels = dict()
for k,v in cluster_color_id.items():
    for el in v:
        if el in cluster_anomaly_bb[k]:
            dict_binary_labels[k] = dict_binary_labels.get(k, []) + ['anomaly']
        else:
            dict_binary_labels[k] = dict_binary_labels.get(k, []) + ['normal']
binary_purity = dict()
for k,v in dict_multi_labels.items():
    binary_purity[k] = collections.Counter(v).most_common(1)[0][1]
purity = sum(list(binary_purity.values()))/n_synopsis
print(f'binary class purity : {purity}')

binary class purity : 0.75


### normalized mutual information 

In [46]:
def entropy(p):
    return p*math.log2(p)

In [64]:
n = sum([len(v) for v in cluster_color_id.values()])
tmp = [el for v in dict_multi_labels.values() for el in v]

P_class = {k:v/n for k, v in dict(collections.Counter(tmp)).items()}
H_class = sum([- entropy(v) for v in P_class.values()])

P_cluster = {k:len(v)/n for k,v in cluster_color_id.items()}
H_cluster = sum([- entropy(v) for v in P_cluster.values()])

H_class_cluster = 0
for k,v in dict_multi_labels.items():
    tmp = {i:j/len(v) for i,j in dict(collections.Counter(v)).items()}
    H_class_cluster +=  P_cluster[k] *sum([- entropy(el) for el in tmp.values()])

I_class_cluster = H_class - H_class_cluster
NMI = 2 * I_class_cluster/(H_class + H_cluster)
print(f'multi class Normalized Mutual Information (NMI) : {NMI}')

multi class Normalized Mutual Information (NMI) : 0.12839000045692578


In [65]:
n = sum([len(v) for v in cluster_color_id.values()])
tmp = [el for v in dict_binary_labels.values() for el in v]

P_class = {k:v/n for k, v in dict(collections.Counter(tmp)).items()}
H_class = sum([- entropy(v) for v in P_class.values()])

P_cluster = {k:len(v)/n for k,v in cluster_color_id.items()}
H_cluster = sum([- entropy(v) for v in P_cluster.values()])

H_class_cluster = 0
for k,v in dict_binary_labels.items():
    tmp = {i:j/len(v) for i,j in dict(collections.Counter(v)).items()}
    H_class_cluster +=  P_cluster[k] *sum([- entropy(el) for el in tmp.values()])

I_class_cluster = H_class - H_class_cluster
NMI = 2 * I_class_cluster/(H_class + H_cluster)
print(f'binary class Normalized Mutual Information (NMI) : {NMI}')

binary class Normalized Mutual Information (NMI) : 0.06834154650464223


### Rand index

In [66]:
tmp = {k:dict(collections.Counter(v)) for k,v in dict_multi_labels.items()}
class_type = set([el for v in dict_multi_labels.values() for el in v])
    
TP_r = 0
for el in tmp.values():
    for v in el.values():
        TP_r += v*(v-1)/2
FP_r = sum([len(v)*(len(v)-1)/2 for k,v in dict_multi_labels.items()]) - TP_r 

TN_r = 0
FN_r = 0
keys = list(tmp.keys())
for i in range(len(keys)-1):
    for j in range(i+1, len(keys)):
        FN_r += sum([tmp[keys[i]].get(k,0)*tmp[keys[j]].get(k,0) for k in class_type])
TN_r = n*(n-1)/2 - FN_r
        
print(f'multi class Rand index : {(TP_r + TN_r)/(TP_r + FP_r + TN_r + FN_r)}')

multi class Rand index : 0.47565026197604793


In [67]:
tmp = {k:dict(collections.Counter(v)) for k,v in dict_binary_labels.items()}
class_type = set([el for v in dict_binary_labels.values() for el in v])
    
TP_r = 0
for el in tmp.values():
    for v in el.values():
        TP_r += v*(v-1)/2
FP_r = sum([len(v)*(len(v)-1)/2 for k,v in dict_binary_labels.items()]) - TP_r 

FN_r = 0
keys = list(tmp.keys())
for i in range(len(keys)-1):
    for j in range(i+1, len(keys)):
        FN_r += sum([tmp[keys[i]].get(k,0)*tmp[keys[j]].get(k,0) for k in class_type])
TN_r = n*(n-1)/2 - (TP_r + FP_r + FN_r)

print(f'binary class Rand index : {(TP_r + TN_r)/(TP_r + FP_r + TN_r + FN_r)}')

binary class Rand index : 0.41347229283172343


### homogenity, completeness, v-measure
not adapted to our clustering case

In [85]:
from sklearn import metrics

In [100]:
labels_true = [0 if el == 'normal' else 1 for v in dict_binary_labels.values() for el in v]
labels_pred = [i for i, (k,v) in enumerate(dict_binary_labels.items()) for el in v]
# labels_pred = [0 if dico_cluster_anomaly_rate[k]<=50 else 1 for k, v in dict_binary_labels.items() for el in v]
print(f'homogenity : {metrics.homogeneity_score(labels_true, labels_pred)}')
print(f'completeness : {metrics.completeness_score(labels_true, labels_pred)}')
print(f'v-measure : {metrics.v_measure_score(labels_true, labels_pred, beta=0.5)}')

homogenity : 0.16890831686961605
completeness : 0.047284646414671
v-measure : 0.09093869549829062


In [103]:
metrics.adjusted_rand_score(labels_true, labels_pred)

-0.034617235141413114

In [104]:
metrics.jaccard_score(labels_true, labels_pred)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [105]:
metrics.adjusted_mutual_info_score(labels_true, labels_pred)

0.03843639521020682

In [ ]:
dico_sfd_freq = dict()
for el in cluster_anomaly_bb.values():
    if len(el)>0:
        for v in el:
            dico_sfd_freq[v.split(':')[0]] = dico_sfd_freq.get(v.split(':')[0], 0)+1
dico_sfd_freq

In [ ]:
anomaly_synopsis_idx = sorted([syn_label.index(el) for v in cluster_anomaly_bb.values() for el in v])
anomaly_synopsis = {syn_label.index(el):fields_data[fields_data.sfd_id == int(el.split(':')[0])].anomaly_class.values[0] for k,v in cluster_anomaly_bb.items() if len(v)>0 for el in v}
color=iter(cm.tab10(np.linspace(0,1,10)))
c = {el:next(color) for el in list(set(fields_data.anomaly_class))}

_, ax = plt.subplots(figsize=(12,6), nrows=1, ncols=2)
axis = ax.flatten()

x_l = mdates.date2num(helper.iloc[0].values)
x_r = mdates.date2num(helper.iloc[-1].values)

axis[0].grid(True)
axis[1].grid(True)
for i in range(len(all_synopsis_list)):
    if i in anomaly_synopsis_idx:
        axis[1].plot(all_date_list[i], all_synopsis_list[i], color=c[anomaly_synopsis[i]]) #label_color[str(i)])#
    else:
        axis[0].plot(all_date_list[i], all_synopsis_list[i], color=label_color[str(i)])
axis[0].set_xlim([x_l, x_r])
axis[0].set_ylim([0, 1])
axis[0].set_title(f'All synopsis not covering anomaly with cluster color')
plt.setp(axis[0].get_xticklabels(), rotation=90, fontsize=8)

axis[1].set_xlim([x_l, x_r])
axis[1].set_ylim([0, 1])
axis[1].set_title(f'Synopsis covering anomaly : crop={crop_type} , year={year_choice}')
plt.setp(axis[1].get_xticklabels(), rotation=90, fontsize=8)

plt.tight_layout()
plt.show()

la précision est tellement faible.......

In [ ]:
cluster_color_id['#ffd500']

In [ ]:
cluster_anomaly_bb['#ffd500']

In [ ]:
interest = [1]

cover = np.ma.zeros((raster_ndvi_numpy_test[0].shape))
for i in range(bb_final_list.shape[0]):
    if i in interest:
        bb_pos = bb_final_list[i,0]
        bb_seg = bb_final_list[i,1]
        segments_fz = segments_test.iloc[bb_pos]
        cover[segments_fz == bb_seg] += 1
cover.mask = raster_ndvi_numpy_test[0].mask
plt.figure(figsize=(10,6))
plt.subplot(1,2,1)
f1 = plt.imshow(cover)
plt.colorbar(f1)

plt.subplot(1,2,2)
f2 = plt.imshow(annotation_df.iloc[0]['annotes_numpy'])
plt.colorbar(f2)
plt.show()

print(annotation_df.index)

In [ ]:
plt.close('all')

## Save cluster results in .shp file

In [93]:
all_sfd_bb_df = gpd.read_file(save_path_shp+'clustered/all_sfd_bb_cover_clustered.shp')
all_sfd_bb_df['cluster_id'] = np.nan
all_sfd_bb_df['cluster_color'] = np.nan

cluster_id = {k:i for i, k in enumerate(cluster_color_id.keys())}
for key in label_color_id.keys():
    sfd_id = int(key.split(':')[0])
    BB_id = key.split(':')[1]
    x = all_sfd_bb_df[(all_sfd_bb_df.sfd_id == sfd_id) & (all_sfd_bb_df.BB_id == BB_id)].index[0]
    all_sfd_bb_df.loc[x, 'cluster_color'] = label_color_id[key]
    all_sfd_bb_df.loc[x, 'cluster_id'] = cluster_id[label_color_id[key]]
all_sfd_bb_df

,sfd_id,BB_id,BB_date,BB_pos_img,BB_seg_id,EPSG,layer,path,geometry,cluster_id,cluster_color
0,102006104,18_2,20181020,102006104_Bands_20181020_Sentinel2_Clear_112.tif,2,32614,102006104_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"MULTIPOLYGON (((-99.72163 39.87525, -99.72127 ...",21.0,#00ff0b
1,102006104,9_30,20180615,102006104_Bands_20180615_Landsat8_Clear_121.tif,30,32614,102006104_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"MULTIPOLYGON (((-99.72826 39.87187, -99.72814 ...",21.0,#00ff0b
2,102006104,3_14,20180322,102006104_Bands_20180322_Sentinel2_Clear_127.tif,14,32614,102006104_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"POLYGON ((-99.72477 39.87369, -99.72465 39.873...",21.0,#00ff0b
3,102006104,0_121,20180302,102006104_Bands_20180302_Sentinel2_Clear_130.tif,121,32614,102006104_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"POLYGON ((-99.72656 39.86657, -99.72621 39.866...",21.0,#00ff0b
4,102006104,5_80,20180428,102006104_Bands_20180428_Landsat8_Clear_125.tif,80,32614,102006104_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"POLYGON ((-99.72785 39.86665, -99.72762 39.866...",21.0,#00ff0b
...,...,...,...,...,...,...,...,...,...,...,...
275,102536577,15_21,20181031,102536577_Bands_20181031_Sentinel2_Clear_59.tif,21,32617,102536577_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"POLYGON ((-79.20299 34.54109, -79.20245 34.541...",2.0,grey
276,102536577,17_76,20181110,102536577_Bands_20181110_Sentinel2_Clear_57.tif,76,32617,102536577_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"POLYGON ((-79.20414 34.53813, -79.20393 34.538...",2.0,grey
277,102536577,5_31,20180309,102536577_Bands_20180309_Landsat8_Clear_69.tif,31,32617,102536577_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"MULTIPOLYGON (((-79.20563 34.53996, -79.20541 ...",2.0,grey
278,102536577,12_15,20181012,102536577_Bands_20181012_Landsat8_Clear_62.tif,15,32617,102536577_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"MULTIPOLYGON (((-79.20484 34.54121, -79.20462 ...",2.0,grey


In [94]:
all_sfd_bb_df.dropna(axis='rows', inplace=True)
all_sfd_bb_df

,sfd_id,BB_id,BB_date,BB_pos_img,BB_seg_id,EPSG,layer,path,geometry,cluster_id,cluster_color
0,102006104,18_2,20181020,102006104_Bands_20181020_Sentinel2_Clear_112.tif,2,32614,102006104_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"MULTIPOLYGON (((-99.72163 39.87525, -99.72127 ...",21.0,#00ff0b
1,102006104,9_30,20180615,102006104_Bands_20180615_Landsat8_Clear_121.tif,30,32614,102006104_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"MULTIPOLYGON (((-99.72826 39.87187, -99.72814 ...",21.0,#00ff0b
2,102006104,3_14,20180322,102006104_Bands_20180322_Sentinel2_Clear_127.tif,14,32614,102006104_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"POLYGON ((-99.72477 39.87369, -99.72465 39.873...",21.0,#00ff0b
3,102006104,0_121,20180302,102006104_Bands_20180302_Sentinel2_Clear_130.tif,121,32614,102006104_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"POLYGON ((-99.72656 39.86657, -99.72621 39.866...",21.0,#00ff0b
4,102006104,5_80,20180428,102006104_Bands_20180428_Landsat8_Clear_125.tif,80,32614,102006104_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"POLYGON ((-99.72785 39.86665, -99.72762 39.866...",21.0,#00ff0b
...,...,...,...,...,...,...,...,...,...,...,...
275,102536577,15_21,20181031,102536577_Bands_20181031_Sentinel2_Clear_59.tif,21,32617,102536577_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"POLYGON ((-79.20299 34.54109, -79.20245 34.541...",2.0,grey
276,102536577,17_76,20181110,102536577_Bands_20181110_Sentinel2_Clear_57.tif,76,32617,102536577_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"POLYGON ((-79.20414 34.53813, -79.20393 34.538...",2.0,grey
277,102536577,5_31,20180309,102536577_Bands_20180309_Landsat8_Clear_69.tif,31,32617,102536577_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"MULTIPOLYGON (((-79.20563 34.53996, -79.20541 ...",2.0,grey
278,102536577,12_15,20181012,102536577_Bands_20181012_Landsat8_Clear_62.tif,15,32617,102536577_2018_scale_5_alpha_0,C:/Users/zwg/Stage GEOSYS/Codes/pinnot_data/va...,"MULTIPOLYGON (((-79.20484 34.54121, -79.20462 ...",2.0,grey


In [95]:
all_sfd_bb_df.to_file(save_path_shp+'clustered/all_sfd_bb_cover_clustered.geojson', driver='GeoJSON')   

In [96]:
plt.close('all')

## Visualize evolution graph

In [102]:
import operator
import copy
from utilefunc.get_palette import get_palette
from evolution_graph import evolution_graph_ndvi

In [97]:
# Parameter settings
## segmentation parameters 
scale=5        # controls the number of produced segments as well as their size. Higher scale means less and larger segments. 
sigma=0          # diameter of a Gaussian kernel, used for smoothing the image prior to segmentation.
min_size=5      # minimum size of the segment

## BB selection and graph construction parameters
alpha = 0.4
t1 = 0.1
t2 = 0
direction=1  # 0 : from small to big; 1: from big to small

In [98]:
## general data informations
fields_data_fpath = 'pinnote_anomaly_info/annotations_valid_dates_final.csv'
fields_data = pd.read_csv(fields_data_fpath, sep=';', )
sfd_ids = fields_data['sfd_id'].unique()  # all season field ids to process
data_path = 'data_images_2017_2020/'  # path for image time series, each stocked in a file folder named with the sfd_id

# save paths
save_path_df = 'variables/raster_df/scale_{0}/'.format(scale)
if direction == 0:
    save_path_bb = 'variables/BB_evolution_graph/BB_small2big/scale_{0}_alpha_{1}_t1_{2}_t2_{3}/'.format(scale, alpha, t1, t2)
    save_path_shp = 'variables/sfd_bbs_cover/BB_small2big/scale_{0}_alpha_{1}_t1_{2}_t2_{3}/'.format(scale, alpha, t1, t2)
if direction == 1:
    save_path_bb = 'variables/BB_evolution_graph/BB_big2small/scale_{0}_alpha_{1}_t1_{2}_t2_{3}/'.format(scale, alpha, t1, t2)
    save_path_shp = 'variables/sfd_bbs_cover/BB_big2small/scale_{0}_alpha_{1}_t1_{2}_t2_{3}/'.format(scale, alpha, t1, t2)

In [99]:
sfd_id_choice = 102424640

In [100]:
year_choice = fields_data[fields_data.sfd_id == sfd_id_choice].year.unique()[0]
raster_df = pd.read_pickle(save_path_df+'{0}_{1}_scale_{2}_raster_seg_df.pkl'.format(sfd_id_choice,year_choice,scale))

segments_test = raster_df['segments_fz'].iloc[(raster_df.index >= datetime.datetime(year_choice,1,1)) & (raster_df.index <= datetime.datetime(year_choice,12,31))]
raster_ndvi_numpy_test = raster_df['raster_ndvi_numpy'].iloc[(raster_df.index >= datetime.datetime(year_choice,1,1)) & (raster_df.index <= datetime.datetime(year_choice,12,31))]

bb_final_list = np.load(save_path_bb+'{0}_{1}_scale_{2}_alpha_{3}_t1_{4}_t2_{5}_final_bb.npy'.format(sfd_id_choice,year_choice,scale,alpha,t1,t2), allow_pickle=True)

In [103]:
## Evolution graph of each BB
evolution_graph_ndvi(sfd_id_choice, year_choice, bb_final_list, segments_test, raster_ndvi_numpy_test, alpha, t1, t2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
plt.close('all')
## Evolution graph temporal coverage with NDVI values
fig = plt.figure(figsize=(3*(bb_final_list.shape[0]+1),3*len(segments_test))) #figsize=(20,50))

for j in range(len(segments_test)):
    date_choice = segments_test.index[j].strftime('%Y-%m-%d')
    raster_ndvi_numpy = raster_df.loc[date_choice, 'raster_ndvi_numpy']
    
    plt.subplot(len(segments_test), bb_final_list.shape[0]+1, j*(bb_final_list.shape[0]+1)+bb_final_list.shape[0]+1)
    cmap, norm = get_palette(raster_ndvi_numpy)
    f2 = plt.imshow(raster_ndvi_numpy, cmap=cmap, norm=norm)
    plt.colorbar(f2, label='NDVI value')
    plt.title(f"NDVI image : {date_choice}",fontsize=6)   
        
for i in range(bb_final_list.shape[0]):
    dico_bb = dict()
    for x in bb_final_list[i,4]:
        dico_bb[x[0]] = dico_bb.get(x[0], []) + [x[1]]
    dico_bb[bb_final_list[i,0]] = [bb_final_list[i,1]]
    dico_bb = dict(sorted(dico_bb.items(), key=operator.itemgetter(0)))
    BB_date = segments_test.index[bb_final_list[i,0]].strftime('%Y-%m-%d')

    for idx in range(len(segments_test)):

        date_choice = segments_test.index[idx].strftime('%Y-%m-%d')
        segments_fz = raster_df.loc[date_choice, 'segments_fz']
        raster_ndvi_numpy = raster_df.loc[date_choice, 'raster_ndvi_numpy']
        bb_ndvi = copy.deepcopy(raster_ndvi_numpy)
        coordinate = np.ma.zeros(segments_fz.shape)

        if idx in dico_bb.keys():
            for ind in dico_bb[idx]:
                coordinate.mask += (segments_fz == ind)
            bb_ndvi.mask = bb_ndvi.mask + ~(coordinate.mask)

            title = f"{date_choice} \n Covered by BB {bb_final_list[i,0]}-{bb_final_list[i,1]} ({BB_date})"
            if np.all(bb_ndvi.mask):
                title = f"{date_choice} \n Totally covered by BB {bb_final_list[i,0]}-{bb_final_list[i,1]} ({BB_date})"
                bb_ndvi.mask = raster_ndvi_numpy.mask
            plt.subplot(len(segments_test),bb_final_list.shape[0]+1, idx*(bb_final_list.shape[0]+1)+i+1)
            cmap, norm = get_palette(raster_ndvi_numpy)
            f = plt.imshow(bb_ndvi, cmap=cmap, norm=norm)
            plt.title(title, fontsize=6, y=1.0)

plt.suptitle(f'BB segment of the field {sfd_id_choice} - {year_choice}', y=1)        
plt.tight_layout()
plt.savefig(f'image_results/evolution_graph/scale_{scale}_alpha_{alpha}_t1_{t1}_t2_{t2}/{sfd_id_choice}_{year_choice}_evolution_graph_alpha_{alpha}_t1_{t1}_t2_{t2}.png', format='png')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
fig = plt.figure(figsize=(3*(bb_final_list.shape[0]+1),3*len(segments_test))) 

for j in range(len(segments_test)):
    date_choice = segments_test.index[j].strftime('%Y-%m-%d')
    raster_ndvi_numpy = raster_df.loc[date_choice, 'raster_ndvi_numpy']
    
    plt.subplot(len(segments_test), bb_final_list.shape[0]+1, j*(bb_final_list.shape[0]+1)+bb_final_list.shape[0]+1)
    cmap, norm = get_palette(raster_ndvi_numpy)
    f2 = plt.imshow(raster_ndvi_numpy, cmap=cmap, norm=norm)
    plt.colorbar(f2, label='NDVI value')
    plt.title(f"NDVI image : {date_choice}",fontsize=6)   
        
for i in range(bb_final_list.shape[0]):
    dico_bb = dict()
    for x in bb_final_list[i,4]:
        dico_bb[x[0]] = dico_bb.get(x[0], []) + [x[1]]
    dico_bb[bb_final_list[i,0]] = [bb_final_list[i,1]]
    dico_bb = dict(sorted(dico_bb.items(), key=operator.itemgetter(0)))
    BB_date = segments_test.index[bb_final_list[i,0]].strftime('%Y-%m-%d')

    for idx in range(len(segments_test)):

        date_choice = segments_test.index[idx].strftime('%Y-%m-%d')
        segments_fz = raster_df.loc[date_choice, 'segments_fz']
        raster_ndvi_numpy = raster_df.loc[date_choice, 'raster_ndvi_numpy']
        coordinate = np.ma.zeros(segments_fz.shape)

        if idx in dico_bb.keys():
            for ind in dico_bb[idx]:
                coordinate[segments_fz == ind] =100
            coordinate.mask = raster_ndvi_numpy.mask

            title = f"Covered by BB {bb_final_list[i,0]}-{bb_final_list[i,1]} ({BB_date})"
            cmap = 'RdYlGn'
            if coordinate.min() == 100:
                title = f"Totally covered by BB {bb_final_list[i,0]}-{bb_final_list[i,1]} ({BB_date})"
                cmap = colors.ListedColormap(['green'])
            plt.subplot(len(segments_test),bb_final_list.shape[0]+1, idx*(bb_final_list.shape[0]+1)+i+1)
            plt.imshow(coordinate, cmap=cmap)
            plt.title(title, fontsize=6, y=1.0)
            
plt.tight_layout()
plt.suptitle(f'BB segment of the field {sfd_id_choice} - {year_choice}', y=1)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Generate and save all evolution graphs

In [5]:
for sfd_id_choice in sfd_ids:
    year_choice = fields_data[fields_data.sfd_id == sfd_id_choice].year.unique()[0]
    raster_df = pd.read_pickle(save_path_df+'{0}_{1}_scale_{2}_raster_seg_df.pkl'.format(sfd_id_choice,year_choice,scale))
    
    segments_test = raster_df['segments_fz'].iloc[(raster_df.index >= datetime.datetime(year_choice,1,1)) & (raster_df.index <= datetime.datetime(year_choice,12,31))]
    raster_ndvi_numpy_test = raster_df['raster_ndvi_numpy'].iloc[(raster_df.index >= datetime.datetime(year_choice,1,1)) & (raster_df.index <= datetime.datetime(year_choice,12,31))]

    bb_final_list = np.load(save_path_bb+'{0}_{1}_scale_{2}_alpha_{3}_t1_{4}_t2_{5}_final_bb.npy'.format(sfd_id_choice,year_choice,scale,alpha,t1,t2), allow_pickle=True)

    ## Evolution graph of each BB
    evolution_graph_ndvi(sfd_id_choice, year_choice, bb_final_list, segments_test, raster_ndvi_numpy_test, alpha, t1, t2)
    
    ## Evolution graph temporal coverage with NDVI values
    fig = plt.figure(figsize=(3*(bb_final_list.shape[0]+1),3*len(segments_test))) #figsize=(20,50))
    for j in range(len(segments_test)):
        date_choice = segments_test.index[j].strftime('%Y-%m-%d')
        raster_ndvi_numpy = raster_df.loc[date_choice, 'raster_ndvi_numpy']

        plt.subplot(len(segments_test), bb_final_list.shape[0]+1, j*(bb_final_list.shape[0]+1)+bb_final_list.shape[0]+1)
        cmap, norm = get_palette(raster_ndvi_numpy)
        f2 = plt.imshow(raster_ndvi_numpy, cmap=cmap, norm=norm)
        plt.colorbar(f2, label='NDVI value')
        plt.title(f"NDVI image : {date_choice}",fontsize=6)   

    for i in range(bb_final_list.shape[0]):
        dico_bb = dict()
        for x in bb_final_list[i,4]:
            dico_bb[x[0]] = dico_bb.get(x[0], []) + [x[1]]
        dico_bb[bb_final_list[i,0]] = [bb_final_list[i,1]]
        dico_bb = dict(sorted(dico_bb.items(), key=operator.itemgetter(0)))
        BB_date = segments_test.index[bb_final_list[i,0]].strftime('%Y-%m-%d')

        for idx in range(len(segments_test)):

            date_choice = segments_test.index[idx].strftime('%Y-%m-%d')
            segments_fz = raster_df.loc[date_choice, 'segments_fz']
            raster_ndvi_numpy = raster_df.loc[date_choice, 'raster_ndvi_numpy']
            bb_ndvi = copy.deepcopy(raster_ndvi_numpy)
            coordinate = np.ma.zeros(segments_fz.shape)

            if idx in dico_bb.keys():
                for ind in dico_bb[idx]:
                    coordinate.mask += (segments_fz == ind)
                bb_ndvi.mask = bb_ndvi.mask + ~(coordinate.mask)

                title = f"{date_choice} \n Covered by BB {bb_final_list[i,0]}-{bb_final_list[i,1]} ({BB_date})"
                if np.all(bb_ndvi.mask):
                    title = f"{date_choice} \n Totally covered by BB {bb_final_list[i,0]}-{bb_final_list[i,1]} ({BB_date})"
                    bb_ndvi.mask = raster_ndvi_numpy.mask
                plt.subplot(len(segments_test),bb_final_list.shape[0]+1, idx*(bb_final_list.shape[0]+1)+i+1)
                cmap, norm = get_palette(raster_ndvi_numpy)
                f = plt.imshow(bb_ndvi, cmap=cmap, norm=norm)
                plt.title(title, fontsize=6, y=1.0)

    plt.suptitle(f'BB segment of the field {sfd_id_choice} - {year_choice} (scale={scale} alpha={alpha} t1={t1} t2={t2})', y=1)
    plt.tight_layout()
    plt.savefig(f'image_results/evolution_graph/scale_{scale}_alpha_{alpha}_t1_{t1}_t2_{t2}/{sfd_id_choice}_{year_choice}_evolution_graph_alpha_{alpha}_t1_{t1}_t2_{t2}.png', format='png')
    plt.show()
    plt.close('all')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …